# Playing with PyPI (data)

Many thanks to Seth M. Larson for making these available: https://github.com/sethmlarson/pypi-data

## Download data

Note that `DuckDB` supports reading directly from remote parquet files (and performs column-pruning and other optimizations to reduce network load) but we'll be executing several queries against these data and so we're better off downloading the files to begin with.

In [ ]:
from pathlib import Path

filenames = [
    "deps.parquet",
    "maintainers.parquet",
    "package_urls.parquet",
    "packages.parquet",
    "scorecard_checks.parquet",
    "wheels.parquet",
]

folder = Path("pypi")
folder.mkdir(exist_ok=True)

for filename in filenames:
    path = folder / filename
    if not path.exists():
        import urllib.request

        urllib.request.urlretrieve(
            f"https://storage.googleapis.com/ibis-tutorial-data/pypi/{filename}",
            path,
        )

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

In [ ]:
con = ibis.duckdb.connect()

In [ ]:
for filename in filenames:
    path = folder / filename
    con.read_parquet(path, table_name=filename.split(".")[0])

In [ ]:
con.list_tables()

In [ ]:
deps = con.tables.deps

deps

In [ ]:
maintainers = con.tables.maintainers

maintainers

In [ ]:
packages = con.tables.packages

packages

## Using `.sql` as an escape hatch

Ibis is pretty good about abstracting away the underlying execution engine, but sometimes you _want_ to talk directly to the engine.

Maybe you:
- have an existing SQL query from a data engineer that you want to build on
- need to access a backend-specific function that isn't exposed in Ibis
- really want to write some SQL

If you do, then you can call `.sql` on a connection (or an existing expression!) and write out your SQL query.

(You can even pass a `dialect` flag if you are more comfortable writing SQL in a dialect different than the one your engine expects!)

### Looking for typo squatters

DuckDB has a `jaccard` function for doing Jaccard similarity checks on strings.  But there's no `jaccard` method in `ibis` -- what to do?  

We'll write a short SQL query:

In [ ]:
# packages.name.jaccard("pandas")  # <--this is not a thing (yet!)

In [ ]:
expr = packages.sql(
    "SELECT name, version, jaccard(packages.name, 'pandas') as sim from packages",
    dialect="duckdb",
)  # the dialect keyword is redundant here

expr

We should probably sort that `sim` column to see which packages have higher similarity.

Even though we called out to `sql` for the Jaccard function call, we don't need to stay in SQL-land moving forward, just chain as usual!

In [ ]:
expr.order_by(_.sim.desc())

### Exercise 1

Try swapping out `jaccard` for a different similarity function available in DuckDB (https://duckdb.org/docs/sql/functions/char#text-similarity-functions) -- or you can try looking for similar PyPI usernames using the `maintainers` table!

In [ ]:
%load solutions/nb03-ex01-jaro.py

In [ ]:
%load solutions/nb03-ex01-maint.py

## Integration with other tools

Plotting data is quite helpful!  Recent support for the `__dataframe__` protocol has made it easier than ever to use Ibis with various plotting libraries.

### Histogram of maintainer count

We'll start by computing the number of packages that have `n` maintainers, as defined by having write permissions on PyPI.

In [ ]:
maintainer_counts = (
    maintainers.group_by("package_name")
    .agg(maintainers=_.count())
    .group_by("maintainers")
    .agg(count=_.count())
    .order_by(_.maintainers)
)

maintainer_counts

We'll plot a histogram of the maintainer counts using `altair` -- but we'll need to install a dev version for this to work:

In [ ]:
!pip install git+https://github.com/altair-viz/altair

In [ ]:
import altair as alt

alt.__version__

In [ ]:
chart = (
    alt.Chart(maintainer_counts)
    .mark_bar()
    .encode(x="maintainers", y=alt.Y("count", scale=alt.Scale(type="log")))
)

chart

### Exercise 2

What's the deal with that spike at 12 maintainers?

**Note**: if you would like to return more rows in interactive mode, you can set the row-count e.g. `ibis.options.repr.interactive.max_rows = 20`

In [ ]:
%load solutions/nb03-ex02-counts.py

In [ ]:
%load solutions/nb03-ex02-ftw.py

In [ ]:
%load solutions/nb03-ex02-ftw-maintainers.py

## UDFs

Modern SQL engines have a _lot_ of features, but there's always going to be _something_ you want to do that isn't built-in.  Maybe you have a text classifier you want to run over a dataset?

We'll tackle something a little less ML-related, and figure out if we need to update our PyPI data pull.

In [ ]:
pp = (
    packages.order_by(_.downloads.desc())
    .select("name", "version", "downloads")
    .limit(10)
)

In [ ]:
pp

In [ ]:
from ibis.expr.operations import udf

In [ ]:
import json

import requests


@udf.scalar.python
def check_version(name: str, version: str) -> bool:
    r = requests.get(f"https://pypi.org/pypi/{name}/json")
    return (
        version == sorted(list(json.loads(r.content.decode())["releases"].keys()))[-1]
    )

In [ ]:
pp.mutate(is_latest=check_version(pp.name, pp.version))

### Exercise 3

Above we used a UDF to look up whether the version of a given package in our dataset was the latest release on PyPI. 

Let's add another UDF to check whether any of the packages in our top-10 downloads have known vulnerabilities.

The API endpoint for checking vulnerabilities is `"https://pypi.org/pypi/{name}/{version}/json"` -- if you load in the JSON from that `GET`, there is a `vulnerabilities` key that will be a list of known vulnerabilities.

Write a UDF that grabs this information, and if it is present, pull out the `aliases` and `fixed_in` subfields.

Here is a sample API response for a known vulnerability:

```
GET /pypi/Django/3.0.2/json HTTP/1.1
Host: pypi.org
Accept: application/json

{
    "info": {},
    "last_serial": 12089094,
    "releases": {},
    "urls": [],
    "vulnerabilities": [
        {
            "aliases": [
                "CVE-2021-3281"
            ],
            "details": "In Django 2.2 before 2.2.18, 3.0 before 3.0.12, and 3.1 before 3.1.6, the django.utils.archive.extract method (used by \"startapp --template\" and \"startproject --template\") allows directory traversal via an archive with absolute paths or relative paths with dot segments.",
            "summary": "A shorter summary of the vulnerability",
            "fixed_in": [
                "2.2.18",
                "3.0.12",
                "3.1.6"
            ],
            "id": "PYSEC-2021-9",
            "link": "https://osv.dev/vulnerability/PYSEC-2021-9",
            "source": "osv",
            "withdrawn": null
        },
    ]
}
```

In [ ]:
%load solutions/nb03-ex03.py

## More exercises / questions:

### What maintainers have the most downloads?

In [ ]:
top_maintainers_by_downloads = (
    maintainers.join(packages, [("package_name", "name")])
    .group_by("name")
    .aggregate(downloads=_.downloads.sum())
    .select("name", "downloads")
    .order_by(ibis.desc("downloads"))
    .limit(10)
)

top_maintainers_by_downloads

### What packages depend on ibis-framework

In [ ]:
ibis_dependents = (
    deps.filter(_.dep_name == "ibis-framework").select("package_name").distinct()
)

ibis_dependents

### What packages depend on things I maintain?

In [ ]:
my_dependents = (
    deps.join(maintainers, _.dep_name == maintainers.package_name)
    .filter(_.name == "gforsyth")
    .select(package="dep_name", dependent="package_name")
    .distinct()
)

my_dependents

### What are the top pytest extensions?

In [ ]:
top_pytest_extensions = (
    deps.filter(_.dep_name.startswith("pytest-"))
    .group_by("dep_name")
    .agg(dep_count=_.package_name.nunique())
    .order_by(_.dep_count.desc())
    .limit(10)
)

top_pytest_extensions

### What packages are the most depended on

In [ ]:
most_dependents = (
    deps.group_by("dep_name")
    .agg(dep_count=_.package_name.nunique())
    .order_by(ibis.desc("dep_count"))
    .limit(10)
)

most_dependents

### What are the most common package prefixes?

In [ ]:
common_prefixes = (
    maintainers.group_by("package_name")
    .agg(maintainers=_.count())
    .filter(_.maintainers == 12)
    .package_name.re_extract(r"^(\w*)-?", 1)
    .name("prefix")
    .topk(5)
)

common_prefixes

### Find the top 20 most depended on packages that have only one maintainer

In [ ]:
bus_factor_1 = (
    maintainers.group_by("package_name")
    .agg(maintainer_count=_.count())
    .filter(_.maintainer_count == 1)
    .join(maintainers, "package_name")
    .join(
        (
            deps.select("package_name", "dep_name")
            .distinct()
            .group_by("dep_name")
            .agg(dep_count=_.count())
        ),
        [("package_name", "dep_name")],
    )
    .select("package_name", "name", "dep_count")
    .order_by(ibis.desc("dep_count"))
    .limit(10)
)

bus_factor_1

In [ ]:
deps.select("package_name", "dep_name").distinct().group_by("dep_name").agg(
    dep_count=_.count()
)

In [ ]:
deps.group_by("dep_name").agg(dep_count=_.package_name.nunique())

### pypi users who have the most distinct collaborators

In [ ]:
most_collaborators = (
    maintainers.join(maintainers, "package_name")
    .select("name", "name_right")
    .filter(_.name != _.name_right)
    .distinct()
    .group_by("name")
    .agg(n_collaborators=_.count())
    .order_by(ibis.desc("n_collaborators"))
    .limit(10)
)

most_collaborators

In [ ]:
maintainers.join(maintainers, "package_name").filter(_.name != _.name_right).group_by(
    "name"
).agg(n_collaborators=_.count()).order_by(_.n_collaborators.desc())

### Finding the most popular transitive dependencies

We can do this by using `.sql` to wrap a recursive CTE, then query it like a normal Ibis table. The recursive CTE will produce a table with a row for each package `package` and its recursive 

In [ ]:
transitive_deps = con.sql(
    """
    WITH RECURSIVE
    direct_deps(package, dependency) AS (
      SELECT
        package_name,
        dep_name
      FROM deps
      WHERE
        extra IS NULL
    ),
    transitive_deps(package, intermediate, dependency) AS (
      SELECT
        package,
        package,
        dependency
      FROM direct_deps
      UNION
      SELECT
        transitive_deps.package,
        direct_deps.package,
        direct_deps.dependency
      FROM direct_deps
      JOIN transitive_deps
        ON direct_deps.package = transitive_deps.dependency
    )
    SELECT package, dependency FROM transitive_deps
    """,
    schema={"package": "string", "dependency": "string"},
)

In [ ]:
top_20_transitive_deps = (
    transitive_deps.group_by("dependency")
    .agg(n_dependents=_.package.nunique())
    .order_by(ibis.desc("n_dependents"))
    .limit(20)
)

In [ ]:
top_20_transitive_deps

In [ ]:
packages.last_uploaded_at.day_of_week.full_name()

In [ ]:
release_days = (
    packages.dropna("last_uploaded_at")
    .group_by(_.last_uploaded_at.day_of_week.index().name("day"))
    .count()
)

release_days

In [ ]:
release_days.mutate(
    release_days["day"]
    .case()
    .when(0, "Sunday")
    .when(1, "Monday")
    .when(2, "Tuesday")
    .when(3, "Wednesday")
    .when(4, "Thursday")
    .when(5, "Friday")
    .when(6, "Saturday")
    .else_("NaD")
    .end()
    .name("day_of_week")
)

In [ ]:
chart = (
    alt.Chart(release_days)
    .mark_bar()
    .encode(x="day:Q", y=alt.Y("CountStar():Q", scale=alt.Scale(type="log")))
)

chart